# Do not run without a VPN, you will get blocked by some of these websites.

In [2]:
import re
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import pandas as pd
import numpy as np

In [3]:
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5055.130 Safari/537.36',
    }
r = requests.get('https://www.gpucheck.com/gpu-benchmark-graphics-card-comparison-chart',headers=headers).text
soup=BeautifulSoup(r,'lxml')

In [4]:
A=soup.findAll('td')
Info=re.findall(r'">(.*?)<',str(A))
Prices=re.findall(r'>\$(.*?)<',str(A))

In [5]:
megalist=[]
for i in range(len(Prices)):
    count = 0
    templist = []
    while count < 5:
        templist.append(Info.pop(0))
        if count == 0:
            templist.append(Prices.pop(0))
        count=count+1
    megalist.append(templist)

In [6]:
df = pd.DataFrame(megalist,columns=('GPU','NEWUSPrice','GPUCheck_Index','AVG1080pFPS','AVG1440pFPS','AVG4kFPS'))
df

,GPU,NEWUSPrice,GPUCheck_Index,AVG1080pFPS,AVG1440pFPS,AVG4kFPS
0,RTX 3090,"1,499.0",287,246.5,187.7,117.9
1,RX 6900 XT,999.0,285,250.1,186.9,105.9
2,RTX 3080 Ti,799.0,283,241.8,182,112.6
3,RX 6800 XT,649.0,276,236.1,176.4,100.1
4,RTX 3080,699.0,275,232.4,173,105
...,...,...,...,...,...,...
104,GTX 760,320.0,40,36.9,27.3,14.9
105,GTX 1050,129.0,39,33.3,24.1,14.1
106,R9 270,275.0,38,32.8,21.9,13.7
107,GTX 950,89.0,36,30,21.4,13.2


### We do not need the GPU Index column. We can create a better ranking column by sorting by a performance column...
##### First, we need to change the majority of the columns to numerical.

In [7]:
df.describe(include='all')

,GPU,NEWUSPrice,GPUCheck_Index,AVG1080pFPS,AVG1440pFPS,AVG4kFPS
count,109,109,109,109,109,109
unique,108,86,77,100,94,98
top,GTX 780M,399.0,45,39,53.5,22.4
freq,2,4,4,3,4,3


In [8]:
GPU_=df.copy()
GPU_.drop('GPUCheck_Index',axis=1,inplace=True)
templist=[]

for entry in list(GPU_['NEWUSPrice']):
    a=str(entry).replace(',','')
    a=float(a)
    templist.append(a)
GPU_['NEWUSPrice']=templist
for entry in ['AVG1080pFPS','AVG1440pFPS','AVG4kFPS']:
    templist=[]
    for i in GPU_[entry]:
        a = float(i)
        templist.append(a)
    GPU_[entry]=templist
    
GPU_.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
GPU,109,108,GTX 780M,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NEWUSPrice,109.0,NaN,NaN,NaN,774.0,588.153277,89.0,320.0,549.0,1199.0,2999.0
AVG1080pFPS,109.0,NaN,NaN,NaN,86.801835,50.781998,30.0,50.2,71.4,105.8,250.1
AVG1440pFPS,109.0,NaN,NaN,NaN,64.23578,38.537628,21.4,36.3,52.8,76.5,187.7
AVG4kFPS,109.0,NaN,NaN,NaN,38.490826,23.154239,12.9,21.6,31.5,46.7,117.9


### GPUs sorted by 1080p FPS...

In [10]:
pd.set_option('display.max_rows',500)
GPU_ = GPU_.sort_values('AVG1080pFPS',ascending=False)
GPU_

,GPU,NEWUSPrice,AVG1080pFPS,AVG1440pFPS,AVG4kFPS
1,RX 6900 XT,999.0,250.1,186.9,105.9
0,RTX 3090,1499.0,246.5,187.7,117.9
2,RTX 3080 Ti,799.0,241.8,182.0,112.6
3,RX 6800 XT,649.0,236.1,176.4,100.1
4,RTX 3080,699.0,232.4,173.0,105.0
5,RX 6800,579.0,197.4,147.6,79.7
6,RTX 3070 Ti,599.0,187.2,139.3,89.0
8,RTX 3070,499.0,178.8,133.1,80.8
7,TITAN RTX,2499.0,166.3,127.3,77.8
10,RX 6700 XT,479.0,163.2,122.0,70.6


In [13]:
html = 'https://www.newegg.com/p/pl?d=6900+xt&Order=1&N=4841%204814'
html = requests.get('https://www.newegg.com/p/pl?d=6900+xt&Order=1&N=4841%204814').text

In [14]:
#6900 xt Newegg Prices, finding minimum

soup=BeautifulSoup(html,'lxml')
A=re.findall(r"(.*?)</strong>",str(soup.findAll('strong')))

def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

templist=[]
for entry in A:
    if has_numbers(entry)==True:
        templist.append(entry)

for entry in templist:
    if '<!-- -->' in entry:
        templist.remove(entry)
templist = re.findall(r"\>(.*?)'",str(templist))

for entry in templist:
    A = str(entry).replace(',','')
    A = float(A)
    templist.pop(0)
    templist.append(A)
min(templist)

719.0

## Let's do that, but faster and for every relevant GPU you can buy on Newegg.

In [15]:
list_of_NewPrice_html = ['https://www.newegg.com/p/pl?d=6900+xt&Order=1&N=4841%204814',
'https://www.newegg.com/p/pl?d=RTX+3090&Order=1&LeftPriceRange=800+&N=4814%20601357248%20100007709',
'https://www.newegg.com/p/pl?d=RTX+3080+ti&Order=1&N=601385735%20100007709%204814',
'https://www.newegg.com/p/pl?d=RX+6800+XT&N=4841%20601359422%20100007709%204814&Order=1',
'https://www.newegg.com/p/pl?d=RTX+3080&N=601357247%20100007709%204814&Order=1',
'https://www.newegg.com/p/pl?d=RX+6800&N=4841%204814%20601359427%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RTX+3070+Ti&N=4814%20601386504%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RTX+3070&N=4814%20601357250%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=TITAN+RTX&N=4814%20601326006%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RX+6700+XT&N=4841%204814&Order=1',
'https://www.newegg.com/p/pl?d=rtx+2080+ti&N=4814%20601321492%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RTX+3060+Ti&N=4814%20601359415%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RTX+2080+SUPER&N=4814%20601341621%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=TITAN+V&N=4814&Order=1',
'https://www.newegg.com/p/pl?d=RTX+2080&N=4814%20601321493%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RX+6600+XT&N=4841%204814%20601388251%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=GTX+1080+Ti&N=4814%20601294835%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RTX+2070+SUPER&N=4814%20601341631%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=Radeon+VII&N=4814%20100007709%20601328427&Order=1',
'https://www.newegg.com/p/pl?d=TITAN+Xp&N=4814&Order=1',
'https://www.newegg.com/p/pl?d=rtx+3060&quicklink=true&N=4814%20601361654%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RX+5700+XT&N=4814%20601341484%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RTX+2070&N=4814%20601321556%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RTX+2060+SUPER&N=4814%20601341616%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RX+5700&N=4814%20601341487%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=GTX+1080&N=601194948%204814%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RTX+2060&N=4814%20601327179%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=RX+5600+XT&N=4814%20601350459%20100007709&Order=1',
'np.nan',
'https://www.newegg.com/p/pl?d=RX+Vega+64&N=4814%20100006662&Order=1',
'https://www.newegg.com/p/pl?d=GTX+1070+Ti&N=4814%20601305993%20100007709&Order=1',
'np.nan',
'np.nan',
'https://www.newegg.com/p/pl?d=1660+ti&N=4814%20601329884%20100007709&Order=1',
'np.nan',
'https://www.newegg.com/p/pl?d=RX+Vega+56&N=4814%20100007709&isdeptsrh=1&Order=1',
'https://www.newegg.com/p/pl?d=GTX+1070&N=4814%20601202919%20100007709&Order=1',
'np.nan',
'https://www.newegg.com/p/pl?d=RTX+3050&N=4814%20601399335%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=GTX+1660+SUPER&N=4814%20601346498%20100007709&Order=1',
'https://www.newegg.com/p/pl?d=GTX+1660&N=4814%20601330988%20100007709&Order=1',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'https://www.newegg.com/p/pl?d=GTX+1650+SUPER&N=4814%20601349617%20100007709&Order=1',
'np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan','np.nan']

In [17]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5055.130 Safari/537.36'
}

import time

delays = [3,5,7]
delay = np.random.choice(delays)

def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

list_new_prices_newegg = []

for entry in list_of_NewPrice_html:
    if entry != 'np.nan':
        html=requests.get(entry,headers=headers).text
        time.sleep(delay)
        soup=BeautifulSoup(html,'lxml')
        A=re.findall(r"(.*?)</strong>",str(soup.findAll('strong')))

        templist=[]
        for entry in A:
            if has_numbers(entry)==True:
                templist.append(entry)


        templist = re.findall(r"\>(.*?)'",str(templist))
        templist2 = []
        for entry in templist:
            if '<!-- -->' not in entry:
                if entry != '30 days':
                    if 'Sale Ends in' not in entry:
                        templist2.append(entry)
        templist = templist2
        templist2 = []
        for entry in templist:
            A = str(entry).replace(',','')
            A = float(A)
            templist.pop(0)
            templist.append(A)
        A=min(templist)
    else:
        A=np.nan
    list_new_prices_newegg.append(A)
list_new_prices_newegg

[719.0,
 1099.0,
 889.0,
 639.0,
 739.0,
 599.0,
 639.0,
 549.0,
 3099.0,
 449.0,
 898.0,
 469.0,
 892.0,
 2398.0,
 1097.0,
 319.0,
 934.0,
 898.0,
 2389.0,
 999.0,
 379.0,
 799.0,
 438.0,
 399.0,
 1094.0,
 748.0,
 249.0,
 698.0,
 nan,
 679.0,
 704.0,
 nan,
 nan,
 266.0,
 nan,
 829.0,
 399.0,
 nan,
 324.0,
 236.0,
 229.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 196.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [18]:
used_price_htmls=['https://www.ebay.com/sch/i.html?_from=R40&_nkw=6900+xt&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3090&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3080+ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6800+xt&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3080+-3080Ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6800&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3070+Ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3070+-3060+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=TITAN+RTX+-bridge+-box+-2080&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6700%20XT&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2080+Ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3060+Ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2080+SUPER&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=titan+v&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2080+-block&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6600+XT+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=1080+Ti+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2070+SUPER+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=TITAN+Xp+gpu+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3060+gpu+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/i.html?_from=R40&_nkw=5700+XT+gpu+-bundle+-box+-block+-shroud&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=2070+-bios+-tesla+-fan+-bundle+-box+-block+-shroud&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=2060+SUPER+-bios+-tesla+-fan+-bundle+-box+-block+-shroud&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=Radeon+Rx+5700&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1080+-fan+-shroud+-waterblock+-block&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=2060+-fan+-shroud+-waterblock+-block&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=5600+XT+-box+-fan+-shroud+-waterblock+-block&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=gpu+Vega+64+-water+-without&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1070+ti+-650&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=R9+295X2&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=GTX+TITAN+X+maxwell&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1660+Ti&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=RX+Vega+56+-watercool+-fans+-vbios&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1070+-box+-cooler&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=RTX+3050&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1660+SUPER&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1660&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=980ti&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=RX+590+-box+-fan+-fans&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+FURY+X&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=gtx+980&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=5500+XT&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=1650+SUPER&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=RX+580+-fan+-box&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'np.nan',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=GTX+TITAN&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=GTX+1060+6gb+-box+-fan+-fans&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=GTX+1060&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=GTX+970&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+290X&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=rx+480+-material&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=rx+570+-box+-fanless&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+390+-backplate+-accelerator+-box+-fanless&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+290+-backplate+-accelerator+-box+-fanless&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=1050+Ti+-box&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
'np.nan',
]

In [19]:
link='https://www.ebay.com/sch/i.html?_from=R40&_nkw=6900+xt&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1'

html=requests.get(link,headers=headers).text

In [20]:
soup=BeautifulSoup(html,'lxml')
float(re.findall(r'Amount","value":(.*?),"currency":"USD"}}',str(soup))[0])

675.0

In [21]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5055.130 Safari/537.36'
}

delays = [5,7,9]
delay = np.random.choice(delays)

def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

list_used_prices_ebay = []

for entry in used_price_htmls:
    if entry != 'np.nan':
        html=requests.get(entry,headers=headers).text
        time.sleep(delay)
        soup=BeautifulSoup(html,'lxml')
        A = float(re.findall(r'Amount","value":(.*?),"currency":"USD"}}',str(soup))[0])
        list_used_prices_ebay.append(A)
    else:
        list_used_prices_ebay.append(np.nan)
list_used_prices_ebay

[675.0,
 900.0,
 814.95,
 599.0,
 650.0,
 504.44,
 450.0,
 450.0,
 849.99,
 399.98,
 450.0,
 399.99,
 340.0,
 1449.95,
 340.0,
 278.0,
 279.97,
 300.0,
 nan,
 349.95,
 369.0,
 319.0,
 275.0,
 255.0,
 275.0,
 199.99,
 200.0,
 200.0,
 nan,
 313.49,
 139.99,
 205.02,
 219.0,
 185.0,
 nan,
 215.0,
 139.99,
 nan,
 269.99,
 160.0,
 149.99,
 nan,
 150.0,
 nan,
 nan,
 160.0,
 nan,
 129.0,
 nan,
 nan,
 99.99,
 115.0,
 125.0,
 nan,
 99.99,
 nan,
 nan,
 nan,
 151.99,
 nan,
 120.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 101.0,
 79.99,
 100.0,
 nan,
 75.0,
 89.99,
 115.0,
 75.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 85.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [22]:
GPU_['Newegg_New_Price'] = list_new_prices_newegg
GPU_['Ebay_Used_Price'] = list_used_prices_ebay

GPU_.to_csv('GPU_Prices_And_Performance.csv')

In [23]:
GPU = pd.read_csv('GPU_Prices_And_Performance.csv',index_col=0)

In [24]:
GPU

,GPU,NEWUSPrice,AVG1080pFPS,AVG1440pFPS,AVG4kFPS,Newegg_New_Price,Ebay_Used_Price
1,RX 6900 XT,999.0,250.1,186.9,105.9,719.0,675.00
0,RTX 3090,1499.0,246.5,187.7,117.9,1099.0,900.00
2,RTX 3080 Ti,799.0,241.8,182.0,112.6,889.0,814.95
3,RX 6800 XT,649.0,236.1,176.4,100.1,639.0,599.00
4,RTX 3080,699.0,232.4,173.0,105.0,739.0,650.00
5,RX 6800,579.0,197.4,147.6,79.7,599.0,504.44
6,RTX 3070 Ti,599.0,187.2,139.3,89.0,639.0,450.00
8,RTX 3070,499.0,178.8,133.1,80.8,549.0,450.00
7,TITAN RTX,2499.0,166.3,127.3,77.8,3099.0,849.99
10,RX 6700 XT,479.0,163.2,122.0,70.6,449.0,399.98


### Continued in the test file so nothing has to be rerun, no risk of website ban.